In [1]:
#import libraries

import numpy as np
import sqlite3
import itertools
import sqlalchemy

conn = sqlite3.connect("soccer_thesis.sqlite")
c = conn.cursor()

#set variables
adjperiod = 20
C = 400
midpoint = 2000
initial_score = 2000
initial_string="2000"
initial_home_field_adv = 0.5 *C


#defining model functions
    
    
def getP(ratinga, ratingb, C):
    Pa = 1/(1+np.exp(-1*(ratinga-ratingb)/C))
    return(Pa)


#count number of matches to use for loop later

c.execute('SELECT Count(*) FROM Match')
matchcount = c.fetchall()
matchcount= int(matchcount[0][0])
print(matchcount)

for i in range(matchcount):
    var = i 
    var_1 =i+1
    n_home_team_score_adj = 0
    n_away_team_score_adj = 0
    sum_home_team_score_adj = 0
    sum_away_team_score_adj = 0
    
    print(i)
#get data to calculate

#match info
    c.execute('''SELECT id, home_team_api_id, away_team_api_id, home_field_adv, league_id
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    match_info = c.fetchall()
    match_id = int(match_info[0][0])
    home_team_id = int(match_info[0][1])
    away_team_id = int(match_info[0][2])
    home_field_adv = match_info[0][3]
    match_league_id = match_info[0][4]


#check if there is already a home_field_adv written into the db
    if not home_field_adv:
        c.execute('''Update match Set home_field_adv = ? Where ID = ?''',
          (initial_home_field_adv,match_id,))
        home_field_adv = initial_home_field_adv
    c.execute('''SELECT home_team_goal
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    g_home = c.fetchall()    
    g_home = int(g_home[0][0])
    c.execute('''SELECT away_team_goal
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    g_away = c.fetchall()
    g_away = int(g_away[0][0])
#add game to league gamecounter
    c.execute('''UPDATE League SET games_played = games_played +1 where id = ?''', (match_league_id,))
    
#home team:

    c.execute('''SELECT home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))

    home_team = c.fetchall()
    ###print(home_team)
    
#away team

    c.execute('''SELECT away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))

    away_team = c.fetchall()
    ###print(away_team)
    
#get individual scores + game counts for home team

    c.execute('''SELECT home_player_1_score, home_player_2_score, home_player_3_score, home_player_4_score, home_player_5_score, home_player_6_score, 
            home_player_7_score, home_player_8_score, home_player_9_score, home_player_10_score, home_player_11_score 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))    
    home_team_score= c.fetchall()
    home_team_score =list(itertools.chain.from_iterable(home_team_score))
    
    c.execute('''SELECT home_player_1_gamecount, home_player_2_gamecount, home_player_3_gamecount, home_player_4_gamecount, home_player_5_gamecount, home_player_6_gamecount, 
            home_player_7_gamecount, home_player_8_gamecount, home_player_9_gamecount, home_player_10_gamecount, home_player_11_gamecount 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))      
    home_team_gamecount= c.fetchall()
    home_team_gamecount =list(itertools.chain.from_iterable(home_team_gamecount))
    
    # add default rating if no rating found yet and write it into db and gamecount +1
    for ii in range(len(home_team_score)):
        
            if home_team_gamecount[ii] == None:
                
                home_team_gamecount[ii] =1
                columncount = 'home_player_'+str(ii+1)+'_gamecount'
                c.execute('''Update match SET %s = 1 Where id = ? ''' % (columncount), (match_id,))
                #conn.commit()
                
                
            if home_team_score[ii] == None:
                
                home_team_score[ii] = initial_score
                columnscore = 'home_player_'+str(ii+1)+'_score'
                c.execute('''Update match SET %s = ? Where id = ? ''' % (columnscore), (initial_score,match_id,))
            
#calculate table of scores where players under x games are weighted lower (adjustment period)

            if home_team_gamecount[ii] > adjperiod:
                sum_home_team_score_adj += home_team_score[ii]
                #print(sum_home_team_score_adj)
                n_home_team_score_adj += 1
            else:
                sum_home_team_score_adj += home_team_score[ii]*home_team_gamecount[ii]/adjperiod
                n_home_team_score_adj += home_team_gamecount[ii]/adjperiod
                
            
            #conn.commit()
                
            ###print(home_team_score)
            
    

#get individual scores for away team

    c.execute('''SELECT away_player_1_score, away_player_2_score, away_player_3_score, away_player_4_score, away_player_5_score, away_player_6_score, 
            away_player_7_score, away_player_8_score, away_player_9_score, away_player_10_score, away_player_11_score 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))    
    away_team_score= c.fetchall()
    away_team_score =list(itertools.chain.from_iterable(away_team_score))
    ###print(away_team_score)
    
    
    c.execute('''SELECT away_player_1_gamecount, away_player_2_gamecount, away_player_3_gamecount, away_player_4_gamecount, away_player_5_gamecount, away_player_6_gamecount, 
            away_player_7_gamecount, away_player_8_gamecount, away_player_9_gamecount, away_player_10_gamecount, away_player_11_gamecount 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,)) 
   
    away_team_gamecount= c.fetchall()
    away_team_gamecount =list(itertools.chain.from_iterable(away_team_gamecount))    

    # add default rating if no rating found yet and write it into db and gamecount +1
    for ii in range(len(away_team_score)):
        
            
            if away_team_gamecount[ii] == None:
                away_team_gamecount[ii] = 1  
                columncount = 'away_player_'+str(ii+1)+'_gamecount'
                c.execute('''Update match SET %s = 1 Where id = ? ''' % (columncount), (match_id,))
                #conn.commit()   
            
            if away_team_score[ii] == None:
                away_team_score[ii] = initial_score
                columnscore = 'away_player_'+str(ii+1)+'_score'
                c.execute('''Update match SET %s = ? Where id = ? ''' % (columnscore), (initial_score,match_id,))
                
#calculate table of scores where players under x games are weighted lower (adjustment period)

            if away_team_gamecount[ii] > adjperiod:
                sum_away_team_score_adj += away_team_score[ii]
                n_away_team_score_adj += 1
                
            else:
                sum_away_team_score_adj += away_team_score[ii]*away_team_gamecount[ii]/adjperiod
                n_away_team_score_adj += away_team_gamecount[ii]/adjperiod
                    
            #conn.commit()
                
            ###print(away_team_score)    
    
# calculate team scores    
    if n_home_team_score_adj > 0:    
        home_team_avg = sum_home_team_score_adj/n_home_team_score_adj
    else:
        home_team_avg = initial_score
        
    if n_away_team_score_adj > 0:
        away_team_avg = sum_away_team_score_adj/n_away_team_score_adj
    else:
        away_team_avg = initial_score
#write them into db:

    c.execute('''Update match Set home_team_score = ? Where id = ? ''',
            (home_team_avg, match_id ))
    
    c.execute('''Update match Set away_team_score = ? Where id = ? ''',
            (away_team_avg, match_id ))
    
    
    
    
    
# neural net:

#calculate goal difference to use for determining learning rate later

    g_difference = abs(g_home-g_away)

#determine winner from data and set winner as w1, loser as w2 

    if g_home > g_away:
        w1_home = True
        w2_home = False
        w1 = home_team_avg
        w1_indv = home_team_score
        w1_gamecount = home_team_gamecount
        w1_ids = home_team
        w1_team_id = home_team_id
        
        
        w2 = away_team_avg
        w2_ids = away_team
        w2_indv = away_team_score
        w2_gamecount = away_team_gamecount
        w2_team_id = away_team_id
        
        target_output = 1
    
    elif g_home == g_away:
        w1_home = True
        w2_home = False
        w1 = home_team_avg
        w1_indv = home_team_score
        w1_ids = home_team
        w1_gamecount = home_team_gamecount
        w1_team_id = home_team_id
        
        w2 = away_team_avg
        w2_indv = away_team_score
        w2_ids = away_team
        w2_gamecount = away_team_gamecount
        w2_team_id = away_team_id
        
        target_output = 0.5
    
    else:
        w2_home = True
        w1_home = False
        w1 = away_team_avg
        w1_indv = away_team_score
        w1_ids = away_team
        w1_gamecount = away_team_gamecount
        w1_team_id = away_team_id
        
        w2 = home_team_avg
        w2_indv = home_team_score
        w2_ids = home_team
        w2_gamecount = home_team_gamecount
        w2_team_id = home_team_id
        
        target_output = 1
    


    
#implement team home-field-adv:
    
    if w1_home == True:
        w1 += home_field_adv
    else:
        w2 += home_field_adv

        
#calculate P(w1 beats w2) (Output)

    e_w1_w2 =  geteterm(w1,w2,C)
    print(e_w1_w2)
    P_w1_beats_w2 = getP(e_w1_w2)

    
    P_w2_beats_w1 = 1 - P_w1_beats_w2

    
#calculate delta error for home-field-adv and calculate new hfa
    learning_rate_hfa = 0.01*C
    if g_home > g_away:
        error_hfa = (1 - P_w1_beats_w2)
        
    elif g_home == g_away:
        error_hfa = (0.5 - P_w1_beats_w2)
        
    else:
        error_hfa =-1*(1 - P_w2_beats_w1)
        
    #print(error_hfa)
    delta_hfa = learning_rate_hfa * error_hfa 
    new_home_field_adv = home_field_adv + delta_hfa
    print(new_home_field_adv)
    
    
    
#search for next match in league and add new home field adv

    c.execute('''Select games_played from League Where id = ? ''', (match_league_id,))
    league_games_played = c.fetchall()
    league_games_played = int(league_games_played[0][0])
    #print(league_games_played)
    c.execute('''Select ID from Match Where league_id = ? Order by Date asc Limit 1 Offset ?''',
              (match_league_id, league_games_played,))
    id_of_next_match_date = c.fetchall()
    
    if not id_of_next_match_date:
        c.execute('''Update league Set home_field_adv = ? where ID = ?''', (new_home_field_adv, match_league_id,))
    else:
        id_of_next_match_date = int(id_of_next_match_date[0][0])
        c.execute('''Update match Set home_field_adv = ? Where ID = ?''',
          (new_home_field_adv,id_of_next_match_date,))         
        

        
        
#calculate delta erros for individual players    
    for iii in range(len(w1_indv)):

    
        
#get gamecount
        
        w1_gamecount_i = w1_gamecount[iii]
        
#get expected probability of player w1i winning against w2        
        w1i =w1_indv[iii]
        if w1_home == True:
            w1i_home = w1i + home_field_adv
            e_w1i_w2 = geteterm(w1i_home,w2,C)
            

        else:
            e_w1i_w2 =  geteterm(w1i,w2,C)
        P_w1i_beats_w2 = getP(e_w1i_w2)
        #print(P_w1i_beats_w2)        
        output = P_w1i_beats_w2
        
#set learning rate and calculate error + delta to calculate new rating        

        if w1_gamecount_i <= 20:
            learning_rate = 0.15
        elif w1_gamecount_i <= 50:
            learning_rate = 0.1
        else:    
            learning_rate = 0.05
        

        learning_rate = learning_rate * (1+0.2*(g_difference))*C #multiply learning by factor determined by goal difference and C 
        error_w1i = target_output - output
        delta_w1i = learning_rate * error_w1i
        ###print(delta_w1i)
        new_rating_w1i = delta_w1i + w1i
        #print(new_rating_w1i)
        

        
#search for column and row to write new data into
        

        player_id = int(w1_ids[0][iii])
        ###print(player_id)
        c.execute('''Select ID FROM match 
            Where ? IN (home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11,away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11) ORDER BY DATE ASC LIMIT 1 OFFSET ?''', 
                  (player_id, w1_gamecount_i,))
        
        
        ID_of_next_match = c.fetchall()
        new_gamecount = w1_gamecount_i +1
        
        if len(ID_of_next_match) > 0:   #check if there still are matches with player left uncalculated
               
            ID_of_next_match = ID_of_next_match[0][0]
            ###print(ID_of_next_match)



            c.execute('''Select CASE 
                        WHEN home_player_1 = ? Then 'home_player_1'
                        WHEN home_player_2 = ? Then 'home_player_2'
                        WHEN home_player_3 = ? Then 'home_player_3'
                        WHEN home_player_4 = ? Then 'home_player_4'
                        WHEN home_player_5 = ? Then 'home_player_5'
                        WHEN home_player_6 = ? Then 'home_player_6'
                        WHEN home_player_7 = ? Then 'home_player_7'
                        WHEN home_player_8 = ? Then 'home_player_8'
                        WHEN home_player_9 = ? Then 'home_player_9'
                        WHEN home_player_10 = ? Then 'home_player_10'
                        WHEN home_player_11 = ? Then 'home_player_11'
                        WHEN away_player_1 = ? Then 'away_player_1'
                        WHEN away_player_2 = ? Then 'away_player_2'
                        WHEN away_player_3 = ? Then 'away_player_3'
                        WHEN away_player_4 = ? Then 'away_player_4'
                        WHEN away_player_5 = ? Then 'away_player_5'
                        WHEN away_player_6 = ? Then 'away_player_6'
                        WHEN away_player_7 = ? Then 'away_player_7'
                        WHEN away_player_8 = ? Then 'away_player_8'
                        WHEN away_player_9 = ? Then 'away_player_9'
                        WHEN away_player_10 = ? Then 'away_player_10'
                        WHEN away_player_11 = ? Then 'away_player_11'
                        END
                        FROM MATCH WHERE ID = ? ''', (player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,ID_of_next_match,))



            columnplayer = c.fetchall()
            ###print(columnplayer)
            columnplayer = columnplayer[0][0]
            columnscore = columnplayer + '_score'
            columncount = columnplayer + '_gamecount'
            

#write new data into db            
            
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columnscore), (new_rating_w1i, ID_of_next_match,  ))
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columncount), (new_gamecount, ID_of_next_match,  ))            
            #conn.commit()
        else:
            new_rating_w1i = int(new_rating_w1i) #round to integer
            c.execute('''Update player SET current_score = ? Where player_api_id = ? ''',
                      (new_rating_w1i,player_id,))
           
            c.execute('''Update player SET games_played = ? Where player_api_id = ? ''',
                      (new_gamecount,player_id,))
            
            c.execute('''Update player Set current_team = ? Where player_api_id = ?''',
                     (w1_team_id, player_id,))

    for iii in range(len(w2_indv)):

#get gamecount
        
        w2_gamecount_i = w2_gamecount[iii]         
        

#get expected probability of player w2i winning against w1        
        w2i =w2_indv[iii]
        if w2_home == True:
            w2i_home =w2i + home_field_adv
            e_w1_w2i =  np.exp(-1*(w1-w2i_home))
            e_w1_w2i = geteterm(w1,w2i_home,C)
        else:
            e_w1_w2i =  geteterm(w1,w2i,C)
        P_w1_beats_w2i = getP(e_w1_w2i)     
        #print(P_w1_beats_w2i)
        output = P_w1_beats_w2i
        
#set learning rate and calculate error + delta to calculate new rating        
        
        if w2_gamecount_i <= 20:
            learning_rate = 0.15
        elif w2_gamecount_i <= 50:    
            learning_rate = 0.1
        else: learning_rate = 0.05
        
        
        learning_rate = learning_rate * (1+0.2*(g_difference))*C #multiply learning by factor determined by goal difference and C
        error_w2i =-1*(target_output - output)
        delta_w2i = learning_rate * error_w2i
        ###print(delta_w2i)
        new_rating_w2i = delta_w2i + w2i
        #print(new_rating_w2i)

#search for column and row to write new data into
        
        player_id = int(w2_ids[0][iii])
        ###print(player_id)
        c.execute('''Select ID FROM match 
            Where ? IN (home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11,away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11) ORDER BY DATE ASC LIMIT 1 OFFSET ?''', 
                  (player_id, w2_gamecount_i,))
        
        ID_of_next_match = c.fetchall()
        new_gamecount = w2_gamecount_i+1
        
        if len(ID_of_next_match)  > 0:        #check if there still are matches with player left uncalculated
        
        
            ##print(ID_of_next_match) 
            ID_of_next_match = ID_of_next_match[0][0]
            

            c.execute('''Select CASE 
                        WHEN home_player_1 = ? Then 'home_player_1'
                        WHEN home_player_2 = ? Then 'home_player_2'
                        WHEN home_player_3 = ? Then 'home_player_3'
                        WHEN home_player_4 = ? Then 'home_player_4'
                        WHEN home_player_5 = ? Then 'home_player_5'
                        WHEN home_player_6 = ? Then 'home_player_6'
                        WHEN home_player_7 = ? Then 'home_player_7'
                        WHEN home_player_8 = ? Then 'home_player_8'
                        WHEN home_player_9 = ? Then 'home_player_9'
                        WHEN home_player_10 = ? Then 'home_player_10'
                        WHEN home_player_11 = ? Then 'home_player_11'
                        WHEN away_player_1 = ? Then 'away_player_1'
                        WHEN away_player_2 = ? Then 'away_player_2'
                        WHEN away_player_3 = ? Then 'away_player_3'
                        WHEN away_player_4 = ? Then 'away_player_4'
                        WHEN away_player_5 = ? Then 'away_player_5'
                        WHEN away_player_6 = ? Then 'away_player_6'
                        WHEN away_player_7 = ? Then 'away_player_7'
                        WHEN away_player_8 = ? Then 'away_player_8'
                        WHEN away_player_9 = ? Then 'away_player_9'
                        WHEN away_player_10 = ? Then 'away_player_10'
                        WHEN away_player_11 = ? Then 'away_player_11'
                        END
                        FROM MATCH WHERE ID = ? ''', (player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,ID_of_next_match,))


            columnplayer = c.fetchall()[0][0]
            columnscore = columnplayer + '_score'
            columncount = columnplayer + '_gamecount'
            


            #write new data into db
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columnscore), (new_rating_w2i, ID_of_next_match,  ))
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columncount), (new_gamecount, ID_of_next_match,  ))  
            #conn.commit()
            
        else:
            new_rating_w2i = int(new_rating_w2i) #round to integer
            print(new_rating_w2i)
            c.execute('''Update player SET current_score = ? Where player_api_id = ? ''',
                      (new_rating_w2i,player_id,))
            c.execute('''Update player SET games_played = ? Where player_api_id = ? ''',
                      (new_gamecount,player_id,))
            c.execute('''Update player Set current_team = ? Where player_api_id = ?''',
                     (w2_team_id, player_id,))
        
        


    conn.commit()

print('finished')

21374
0
0.606530659713
201.510162675
1
0.60424507702
203.016778056
2
0.601973445482
204.519857766
3
0.599715655008
204.019413398
4
1.66537201931
202.518684805
5
0.602723509664
204.022933058
6
0.606530659713
201.510162675
7
0.60424507702
203.016778056
1972
8
0.600461151799
205.5236534
9
0.601973445482
204.519857766
10
0.606530659713
199.510162675
11
0.606530659713
201.510162675
12
1.654957629
200.003547294
13
0.60652528088
201.513701633
14
1.65497227113
200.007094561
1955
15
0.606519902136
201.517240564
16
0.604234385167
203.023839327
17
0.601962818833
204.526902474
18
0.633275119414
205.074586521
19
0.62813056184
206.617781094
20
0.674011479624
208.228311219
21
0.562122826939
209.66769311
22
1.64670349339
197.998848692
23
0.609572661809
199.51371702
24
0.607268471946
199.025022648
25
0.63981360521
206.080556136
26
1.57466067086
204.526953347
27
0.599705093048
204.026441597
28
0.60045588497
205.527153714
29
0.531910059774
211.056573759
30
0.636471546859
210.612290504
31
0.590650547726
2

253
0.68647093634
202.012475851
254
1.55003151485
207.763753017
255
1.39383418041
199.969972148
256
0.55742825494
201.401635541
257
2.14202164141
200.128569835
258
2.23634844443
221.892235492
259
1.48434854344
220.282155453
1980
260
0.411452708623
221.44819582
261
1.26885505089
219.685192054
262
0.503450198815
221.024644998
263
0.445279373502
220.257013996
264
0.506720882205
221.602242268
265
0.546742890271
207.177673569
266
1.95502489608
205.824047072
267
0.782745539348
207.580316875
268
1.99273072025
206.243744903
269
0.551313783785
207.665285251
270
0.441822956319
219.55489466
271
0.408950382018
220.715901853
272
0.407734009557
202.055284327
273
0.568896872725
203.50572223
274
1.43749531225
201.864693433
275
1.79029610124
200.431153818
276
0.421719425043
203.198981198
277
0.483270819071
204.502238332
1950
278
1.11488158756
198.237210988
279
2.6778836827
197.149629013
280
0.5943052062
196.640699114
281
1.19367810267
194.81727753
282
0.512616020044
196.172852278
1972
283
0.53911869852

0.292741521914
240.663147246
503
0.443141564705
241.891416393
504
1.20708817725
240.079073759
505
0.485175214441
241.385788845
506
0.981742051144
197.180977451
507
0.660243394281
204.278179438
508
0.817444733646
204.077287204
509
0.464892843105
205.34671205
510
1.42273306814
203.695684105
511
0.471228145506
198.462160514
512
0.320292189022
199.432528109
1973
513
1.14164583746
199.564805664
514
0.474678113236
200.852349323
1920
515
0.622599646295
200.387169444
516
0.581281901732
242.856195611
517
0.388569133174
207.462988609
518
2.09972580487
206.17255189
519
0.700742492031
205.820637935
520
0.514844047309
205.180102094
521
1.66904322285
203.681437717
522
0.580212080373
205.15013192
523
0.968493681236
203.118121333
524
0.68483760543
200.744005283
525
0.567465694855
216.729859157
1912
526
1.3667383213
215.039769482
527
0.366922841527
216.11348873
1975
528
0.317424972927
217.077262697
529
2.99895629635
216.077001703
530
0.582951851662
215.550077087
531
0.5215917001
209.413811613
532
0.510

750
0.60781532796
214.893823271
751
0.615337824671
214.417561068
752
1.15810873261
256.22895747
753
0.549098612318
199.367288781
754
0.349616775612
202.340972446
755
0.735183901328
202.035741285
756
2.30592240906
200.825791532
1924
757
1.24300655925
199.042470844
1956
758
0.814667671309
200.838210492
759
0.781820192047
219.744873256
760
1.19017219383
217.918532838
761
0.808912103777
187.896221344
762
0.54756515687
189.311515956
763
0.806970228652
191.09786576
1938
764
2.60918898946
213.309278844
765
1.17376103299
211.469150174
1916
766
0.298358361386
210.388336649
767
0.501671955914
257.56525986
1967
768
1.37259259981
255.879340415
769
0.97443488918
257.853444285
770
0.345236234148
256.879988844
771
0.627416139153
258.422104899
772
1.38837915604
256.747328937
1936
773
0.574250715511
256.206437603
1971
774
1.93539583405
216.555854608
775
0.452414408801
217.801819688
776
0.649921449867
217.377461849
777
0.655815813535
218.961734226
778
0.980527806676
218.942070586
779
0.738684776004
216.

996
1.03744886542
228.801350279
997
0.187553770179
255.519888282
998
0.989910719586
253.509747847
999
0.525366032553
218.58284574
1000
2.6938037452
217.499951175
1001
1.09390401654
197.474079295
1002
0.899258430624
195.367994133
1003
0.593082007625
214.417227143
1004
0.777355148513
216.166692126
1005
0.415423813845
229.975341588
1006
0.852504004374
231.816101991
1007
0.539188736643
252.910976231
1008
0.49217342734
254.230322678
1009
0.585772017563
255.707891991
1010
2.93895753825
254.69239488
1011
0.356835760673
255.744359373
1012
0.399974213989
256.88716389
1013
1.16070714243
255.035918101
1014
0.567767897269
254.484520023
1015
0.854187268718
217.342671774
1016
0.667295845787
216.94357745
1017
2.1283383409
215.664943375
1018
0.633355121958
217.215996452
1019
0.418641370555
218.396397304
1020
0.887114309136
197.248355709
1021
1.21857323457
218.363731585
1022
0.333130595763
217.363275356
1023
0.679563389124
216.981704034
1024
0.275109225431
217.844717869
1025
0.423097837264
217.03394836

...

1.27140136095
185.087207292
2682
20605
0.532068884107
186.476358628
20606
0.52447152676
187.852498536
1864
20607
0.806527790661
187.638306194
2329
1970
20608
0.491025389581
188.95558863
20609
0.655930318764
188.540028051
1954
20610
1.1209677761
186.654096523
20611
0.270557570503
177.728363234
1989
20612
0.941325061023
175.667914894
20613
0.397030223408
203.451848831
1708
20614
0.607447645875
200.963431862
1935
1792
20615
1.99013634398
199.62570022
2116
20616
1.81610633795
256.571058754
20617
0.266131609938
257.411829517
1886
1782
20618
1.3675845978
129.301953261
2229
2359
2440
20619
0.696063249152
73.2057161529
1912
20620
0.974551803805
71.1799399781
20621
3.30718252263
183.917227557
20622
0.253716391814
184.726713317
20623
0.828415568069
184.539026847
1951
20624
0.558030941348
198.217887818
20625
0.981824516117
151.442593161
1930
20626
1.35810682392
153.746317166
20627
0.253008227503
209.831273995
20628
0.281349925256
210.709566239
2016
20629
0.588489975068
212.191451512
20630
0.78462

20810
1.57886840944
254.280911699
1872
1872
1858
20811
0.484420295298
253.586257083
1958
1932
1952
2008
20812
0.919731133899
129.180776855
1966
20813
0.928622899073
127.106758133
1716
20814
0.385060246664
128.218797115
1990
20815
0.48332861386
127.522159321
1896
20816
2.2076542686
186.809946402
20817
0.280688660179
187.686626797
20818
1.96805988819
186.338945095
2213
20819
0.920456054507
188.256106495
2138
1914
20820
2.31793712915
203.069464703
20821
0.643668007432
200.635883236
1789
20822
1.41716027896
198.981048828
1789
20823
3.08436019263
211.755315423
1917
20824
0.709918624913
188.824253818
2167
20825
2.59054122325
252.472218906
1760
1975
1997
2135
20826
0.403490807175
128.672122841
2146
20827
0.720627303614
189.931372957
1862
1893
20828
0.279594548962
148.425976397
20829
0.493172592897
145.74711662
1736
20830
0.63715737491
143.303857309
2085
1933
20831
0.90045149531
143.199094304
2187
20832
0.742855083453
144.904009625
20833
0.587999273393
144.385116785
2175
20834
0.798447414171
1

0.504688026084
190.233629162
1787
2005
21013
0.454244681765
187.483060396
21014
0.562570431966
127.811023894
21015
0.496278921497
125.137725526
1970
1896
21016
0.303943577124
184.788823212
21017
0.980538545704
186.769170523
21018
0.464144728531
186.037200163
1973
1827
1981
21019
0.483750151412
185.341328492
21020
0.593151593648
203.07062793
2070
21021
1.68342850214
201.579997568
21022
0.275372095884
147.489786084
2360
21023
0.966420099558
145.45563275
21024
0.480935172021
144.754636734
2065
2098
21025
0.43352037082
206.005738487
2079
2154
1775
2090
21026
1.69025689638
204.518891634
21027
2.71691233396
203.442729583
21028
0.270480556241
194.383922651
1981
1883
21029
2.13849565664
193.109426702
2053
21030
0.79982349397
174.749766408
21031
0.413760377935
175.920432624
1996
1929
21032
1.95403732097
174.566353591
1831
21033
1.17486100029
172.727155592
21034
0.137535278214
173.210781132
1855
1918
1825
1899
21035
1.54377721165
173.63831638
2072
2402
21036
0.440516743958
174.861535051
1998
210

1866
21209
1.15057759926
184.309884113
2195
2169
1969
1969
2119
21210
0.602939308627
185.814468361
2131
21211
0.85321672175
204.952688894
21212
1.76541451223
203.506251124
21213
0.60793172778
205.018583314
1955
1955
21214
0.70490847107
149.131569082
2456
21215
0.630700637245
189.675658344
2028
2049
1991
1910
21216
0.229069307611
189.407639689
1950
1981
21217
0.726266029974
187.090499729
2028
1945
21218
0.449090135372
187.05411557
1809
1738
2132
21219
0.96611667775
149.097101827
2089
2008
1950
21220
0.748373272567
148.809260859
21221
0.857516757737
150.655848254
1945
1962
2388
2052
2013
21222
0.488811650035
151.969141712
2144
2264
2008
21223
0.434561196071
149.180832639
1883
1825
1916
1985
21224
1.73718736086
151.719478528
2468
2344
21225
1.94412906751
150.36084244
2091
21226
0.410856530544
200.228310226
1960
2006
1971
2050
21227
1.69524251056
198.744213717
1973
1954
1950
21228
0.37062233513
195.82583139
2011
1967
1803
2038
21229
0.374160647893
194.914963577
2073
2007
21230
0.5846606609

2171
2464
21336
0.188564733836
156.05945741
1904
1941
2042
1951
2009
1823
1804
2032
1852
1924
1862
21337
0.569037290267
155.510123481
2425
2260
2302
2362
2333
2395
2349
2017
2117
2388
2395
21338
1.62438555528
157.985957189
2419
2424
2497
2446
2099
2346
2158
2302
2301
2444
2405
21339
0.381078185224
159.089669316
2006
2039
1912
2034
2100
1938
1997
2079
1879
2061
2292
21340
0.816171689632
160.887234368
2114
2102
2044
2289
2178
2234
2335
2121
2259
2228
2087
21341
1.60764751877
161.353284718
2478
2432
2545
2470
2530
2370
2552
2473
2313
2141
2470
21342
0.720478688955
161.028350369
1985
2136
2012
2037
2006
1980
2006
2305
1895
2001
2037
21343
0.862517035256
160.880719005
1923
2474
2308
2299
2021
2091
2008
2224
1954
2100
2148
21344
1.9653887392
182.258949366
1943
1948
2143
2025
2043
2000
1975
2187
1942
2007
2041
21345
0.567764552156
181.707545844
1945
2047
1845
1874
1982
2096
1820
2047
1757
2007
2000
21346
1.3322163558
179.9924391
1834
1969
1946
1879
1881
1947
1807
1879
1862
2007
1825
21347
1.4

In [3]:
import numpy as np

def getP(ratinga, ratingb,C):
    Pa = 1/(1+np.exp(-1*(ratinga-ratingb)/C))
    return(Pa)

P=getP(2200,2000,400)


0.62245933120185459